# Functional Closures and Decorators

Sometimes we may have "cross-cutting" concerns: functionality which is not related to the "core" functionality of a function:

- logging
- authentication
- locking
- entry of the function into a global registry

Python has a syntax that uses two features of the language (scoping and first-class functions) to enable such operations to be separated from the core function.

First, though, let's see if you can guess what the following code prints:

In [1]:
a = 5
b = 12
c = 30
def foo():
    a = 10
    b = 15
    print(f'1 => a={a}, b={b}, c={c}')  # '1 => a={}, b={}'.format(a, b)   Py 3.5+
    def bar():
        a = 20
        print(f'2 => a={a}, b={b}, c={c}')
    bar()
    print(f'3 => a={a}, b={b}, c={c}')
foo()
print(f'4 => a={a}, b={b}, c={c}')


1 => a=10, b=15, c=30
2 => a=20, b=15, c=30
3 => a=10, b=15, c=30
4 => a=5, b=12, c=30


##  LEGB
* In order to understand closures, let's review the Python scoping rules: LEGB
  * L = local
  * E = enclosing (i.e. `nonlocal`)
  * G = global (i.e. `global`)
  * B = builtin (e.g., len() function)
  
```python
a = 'global scope'

def outer_func():
    b = 'local to outer_func()'
    def inner_func():
        c = 'local to inner_func()'
        print(b, 'enclosing/nonlocal scope')
        print(a, 'global scope')
    return inner_func
```
                
* When a function references a name that is not local, Python first attempts to resolve that name in the enclosing scope
* A *closure* is a nested function which "remembers" a value or values from the enclosing lexical scope even when the program flow is no longer in the enclosing scope

## `global` and `nonlocal` keywords

Python scoping works as described above for *reading* values associated with names

When **assigning** values, assignment is local *unless*

- the value is declared as `global`, in which case it assigns globally
- the value is declared as `nonlocal`, in which case it assigns to the "nearest" enclosing scope

# Closures

In [2]:
def make_adder(x):
    
    def adder(y):
        return x + y # Python uses LEGB to find 'x'
    
    return adder

In [3]:
add39 = make_adder(39)
add39

<function __main__.make_adder.<locals>.adder(y)>

In [4]:
add39(10)

49

In [5]:
add12 = make_adder(12)
add12(10)

22

In [6]:
add39(10)

49

In [7]:
add39.__closure__

(<cell at 0x7f8c242bb160: int object at 0x9552e0>,)

In [8]:
cell = add39.__closure__[0]

In [9]:
cell.cell_contents

39

In [10]:
x = make_adder(39)
x(10)

49

In [11]:
make_adder(39)(10)  # like add(39, 10)

49

In [12]:
add6 = make_adder(6)

In [13]:
print(add6(10), add39(10))

16 49


In [14]:
# let's use repr so we can see the address of the function
# we could use print("%X") as well...
#type(add39)
repr(add39)

'<function make_adder.<locals>.adder at 0x7f8c242d4940>'

In [15]:
repr(add6)

'<function make_adder.<locals>.adder at 0x7f8c242d4310>'

## Building function wrappers

* One case where closures are frequently used is in building function wrappers
* Suppose we want to log each invocation of a function:

In [16]:
def logging(function):
    print('Wrapping {}'.format(function))
    def wrapper(*args, **kwargs):
        print('Calling %r(%r, %r)' % (function, args, kwargs))
        return function(*args, **kwargs)
    return wrapper

In [17]:
def myprinter(x):
    print('The value of x is', x)

In [18]:
myprinter(22)

The value of x is 22


In [19]:
logging_myprinter = logging(myprinter)

Wrapping <function myprinter at 0x7f8c242d4790>


In [20]:
logging_myprinter

<function __main__.logging.<locals>.wrapper(*args, **kwargs)>

In [21]:
myprinter(42)

The value of x is 42


In [22]:
logging_myprinter(42)

Calling <function myprinter at 0x7f8c242d4790>((42,), {})
The value of x is 42


In [23]:
logging_myprinter(5)

Calling <function myprinter at 0x7f8c242d4790>((5,), {})
The value of x is 5


In [24]:
logging_myprinter.__closure__[0].cell_contents

<function __main__.myprinter(x)>

In [25]:
myprinter = logging(myprinter)

Wrapping <function myprinter at 0x7f8c242d4790>


In [26]:
myprinter

<function __main__.logging.<locals>.wrapper(*args, **kwargs)>

In [27]:
myprinter(5)

Calling <function myprinter at 0x7f8c242d4790>((5,), {})
The value of x is 5


In [28]:
myprinter(22)

Calling <function myprinter at 0x7f8c242d4790>((22,), {})
The value of x is 22


In [29]:
def myprinter(x):
    print('The value of x is', x)
myprinter = logging(myprinter)

Wrapping <function myprinter at 0x7f8c242d4e50>


In [30]:
@logging
def myprinter(x):
    print('The value of x is', x)

Wrapping <function myprinter at 0x7f8c242d4dc0>


In [31]:
myprinter(22)

Calling <function myprinter at 0x7f8c242d4dc0>((22,), {})
The value of x is 22


# Non-wrapping decorators

In [32]:
# Flask-like routing

routes = {}  # map url => python function


def route(path):   # "decorator factory" (returns a decorator)
    print(f'Build route({path})')
#     print('Build route({path})'.format(path=path))
#     print('Build route(%s)' % path)
    def decorator(function):   # decorator (takes a single function as input)
        print(f'Call decorator({path})({function})')
        routes[path] = function
        # (no wrapping going on here)
        return function
    return decorator

In [33]:
@route('/')
def get_home():
    return 'Welcome'

# route_home_deco = route('/')
# def get_home():
#     return 'Welcome'
# get_home = route_home_deco(get_home)

@route('/about')
def get_about():
    return 'About us'

@route('/error')
@route('/other_error')
def get_error():
    return '404'

# route_error_decorator = route('/error')
# route_other_error_decorator = route('/other_error')
# def get_error():
#     return '404'
# get_error = route_other_error_decorator(get_error)
# get_error = route_error_decorator(get_error)

Build route(/)
Call decorator(/)(<function get_home at 0x7f8c242d43a0>)
Build route(/about)
Call decorator(/about)(<function get_about at 0x7f8c24299160>)
Build route(/error)
Build route(/other_error)
Call decorator(/other_error)(<function get_error at 0x7f8c242d4430>)
Call decorator(/error)(<function get_error at 0x7f8c242d4430>)


In [34]:
get_home

<function __main__.get_home()>

In [35]:
routes

{'/': <function __main__.get_home()>,
 '/about': <function __main__.get_about()>,
 '/other_error': <function __main__.get_error()>,
 '/error': <function __main__.get_error()>}

In [36]:
def run_request(routes, url):
    function_to_call = routes[url]
    return function_to_call()

In [37]:
run_request(routes, '/')

'Welcome'

In [38]:
run_request(routes, '/about')

'About us'

In [39]:
run_request(routes, '/error')

'404'

Wrapping order matters...

```python
def with_api_client(func):
    def wrapper(*args, **kwargs):
        cli = get_client_function()
        return func(cli, *args, **kwargs)
    return wrapper


@app.route('/patient/<patientId>')
@with_api_client
def get_patients(cli, patientId):
    ...

@with_api_client
@app.route('/patient/<patientId>')
def get_patients(cli, patientId):
    ...
```

```python
@EXPR0  # (
@EXPR1  # (
def FUNC():
    ...
# ))
```
    
... means nothing more than ...

```python
_temp0 = EXPR0
_temp1 = EXPR1
def FUNC():
    ...
FUNC = _temp1(FUNC)
FUNC = _temp0(FUNC)
```

In [40]:
def my_decorator_factory(a, b, c=5):   # decorator factory
    def decorator(func):               # decorator
        def wrapper(*args, **kwargs):  # wrapper
            print(f'Calling wrapper with {a}, {b}, {c}')
            return func(*args, **kwargs)
        return wrapper
    return decorator

@my_decorator_factory('foo', 'bar')
def somefunc(x, y):
    return x + y

In [41]:
somefunc(1, 2)

Calling wrapper with foo, bar, 5


3

# Auth psuedo-code

```python
def authorize_or_403(perms):
    ...
    
def require_authorized(perms):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            authorize_or_403(perms)
            return func(*args, **kwargs)
        return wrapper
    return decorator

@route('/protected')
def get_some_protected_thing():
    authorize_or_403('Patient.read')
    ...

@route('/protected2')
@require_authorized('Patient.read')
def get_some_other_protected_thing():
    ...
```

# Preserving introspection

In [42]:
def document_it(func):
    # below is a nested, or inner function
    def wrapper(*args, **kwargs):
        "wrapper docstring"
        print('Running function: {}'.format(func.__name__))
        print('Positional arguments: {}'.format(args))
        print('Keyword arguments: {}'.format(kwargs))
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print('Result: {}'.format(result))
        return result
    
    # document_it() is returning a reference to the inner function
    return wrapper

In [43]:
@document_it
def addit(x, y):
    """I heard I should use docstrings so here ya go"""
    return x + y

In [44]:
help(addit)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
    wrapper docstring



In [45]:
addit

<function __main__.document_it.<locals>.wrapper(*args, **kwargs)>

In [46]:
from functools import wraps

def document_it(func):
    """This is the decorator docstring"""
    # below is a nested, or inner function
    @wraps(func)
    def wrapper(*args, **kwargs):
        """This is the wrapper docstring"""
        print('Running function: {}'.format(func.__name__))
        print('Positional arguments: {}'.format(args))
        print('Keyword arguments: {}'.format(kwargs))
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print('Result: {}'.format(result))
        return result
    
    # document_it() is returning a reference to the inner function
    return wrapper

In [47]:
@document_it
def addit(x, y):
    """I heard I should use docstrings so here ya go"""
    return x + y

In [48]:
help(addit)

Help on function addit in module __main__:

addit(x, y)
    I heard I should use docstrings so here ya go



In [49]:
addit

<function __main__.addit(x, y)>

In [50]:
addit(1, 2)

Running function: addit
Positional arguments: (1, 2)
Keyword arguments: {}
Result: 3


3

# Lab

Open the [Closures Lab][closures-lab]

[closures-lab]: ./closures-lab.ipynb
